# **Task21**
Consider the expression syntax tree discussed here: https://ruslanspivak.com/lsbasi-part7/ . 
Develop a python program that generates a set of random expressions based on the syntax tree definition. 
(e.g., this may be useful to generate synthetic expression dataset)



## The Grammer for Generating the Expressions are:

### Tokens

In [25]:
PLUS, MINUS, MUL, DIV, EOF, LPARAM, RPARAM,INTEGER  = 'PLUS', 'MINUS', 'MUL','DIV', 'EOF', 'LPARAM', 'RPARAM','INTEGER'

### DEFINING THE CLASS TOKEN

In [26]:
class Token:
    def __init__(self,type_,value):                  #calling the initialization function and initializing the values
        self.type_ = type_                           #Storing the type of the token to self.token
        self.value = value                           #Storing the value of the token to self.value

    def __str__(self):
        return f"Token({self.type_},{self.value})"   #Returning the object of the class as a stirng

    def __repr__(self):                              #Retunring the stirng 
        return __str__()

### Defining the class Lexer

In [27]:
class Lexer:
    def __init__(self,text):                        #Calling the init function and passing the text to it
        self.text = text                            #Storing the text to self.text
        # print(self.text)
        self.pos = 0                               #Initializing the position to 0
        self.current_char = self.text[self.pos]    #Storing the 0th index value of the text to the variable current_char
        # print(self.current_char)

    def error(self):                               #defining the error function to raise the exception
        raise Exception("Invalid Character")

    def advance(self):                             #defining the function advance
        self.pos +=1                               #incrementing the character by 1 position
        if self.pos > len(self.text) - 1:          #checking whether the position value is the last value of text
            self.current_char = None               #if yes, making current_char - none
        else:
            self.current_char = self.text[self.pos]  #else, storing the current position of the text to the variable current_char

    def skip_whitespace(self):                        #function to remove whitespace
        while self.current_char is not None and self.current_char.isspace():        #checking the character is not none and isspace
            self.advance()                                                          #calling the advance function

    def integer(self):                                #defining the function integer to merge the 2 or more digit integers
        res = ''                                      #initializing the variable res with blank
        while self.current_char is not None and self.current_char.isdigit():         #checks the current_char is digit or not
            res += self.current_char                                                 #adding the current_char to the variable res
            self.advance()                                                           #calling the function advance
        return int(res)                                                              #returning the res of int
            
            
    def get_next_token(self):                                       #defining the function get_next_token 

        while self.current_char is not None:                        #while loop to check the character is till none

            if self.current_char.isspace():                     #condition to check for white space
                self.skip_whitespace()                             #if yes, calling the function skip_whitespace
                continue
            elif self.current_char.isdigit():                   #condition to check for is digit
                return Token(INTEGER,self.integer())               #if yes, returning the token 
            elif self.current_char == '+':                      #condition for '+'
                self.advance()                                     #if yes, calls the advance function to increment the position  
                return Token(PLUS,'+')                             #returs the token of PLUS 
            elif self.current_char == '-':                      #condition for '-'
                self.advance()                                     #if yes, calls the advance function to increment the position
                return Token(MINUS,'-')                            #returns the token of MINUS
            elif self.current_char == '*':                      #condition to check for '*'
                self.advance()                                     #if yes, calls the advance function to increment the position 
                return Token(MUL,'*')                              #returns the token of MUL
            elif self.current_char == '/':                      #condition to check for '/'
                self.advance()                                     #if yes, calls the advanvce function to increment the position
                return Token(DIV,'/')                              #returns the token of DIV 
            elif self.current_char == '(':                      #condition to check for the '('
                self.advance()                                     #calling the advance function to increment the position  
                return Token(LPARAM,'(')                           #returns the token LPARAM
            elif self.current_char == ')':                      #condition to check for the ')'
                self.advance()                                     #calling the advance function to increment the position
                return Token(RPARAM,')')                           #returns the token of RPARAM
            self.error()                                        #if no condition satisfied calls the error function to raise the exception
        return Token(EOF,None)                               #if the current char is None then , return the token of None

### Defining the class Interpreter

In [28]:
class Interpreter:
    def __init__(self,lexer):                                        #defining the init function to initialize the variables
        self.lexer = lexer                                           #storing the lexer to the variable lexer
        # print(self.lexer)
        self.current_token = self.lexer.get_next_token()             #calling the get_next_token function of lexer class to initialize the current token
        # print(self.current_token)

    def error(self):                               #defining the error function to raise the exception
        raise Exception("Invalid Syntax")

    def eat(self,token_type):                                      #defining the function eat to check for token type and store next token 
        if self.current_token.type_ == token_type:                  #condition to check for the type of the token is same
            self.current_token = self.lexer.get_next_token()         #calling the next token function and storing it in current_token variable
            # print(self.current_token)
        else:                                                        
            self.error()                                             #if the token type mismatched, then call error function

    def factor(self):                                           #defining the function factor 
        token = self.current_token                                 #storing the current_token to the token variable
        # print(token)
        if token.type_ == INTEGER:                                #condition to check if the token type is INTEGER
            self.eat(INTEGER)                                       #if yes, calling the eat function
        elif token.type_ == LPARAM:                               #condition to check if the token type is LPARAM
            self.eat(LPARAM)                                        #if yes calling the eat function
            res = self.expr()                                       #calling the expr function to create a one more expression  
            self.eat(RPARAM)                                        #calling the eat function and passing RPARAM 
            return res                                              #returning the res
            
            
        return token.value                                          #returning the value of the token 
    
    def term(self):                                             #defining the function term   

        res = self.factor()                                        #calling the function factor and storing it to the variable res
        # print(res)

        while self.current_token.type_ in (MUL, DIV):              #checking if the type of the current token is MUL or DIV
            token = self.current_token                               #if yes, stores the token to the token variable
            if token.type_ == MUL:                                   #check got the token type is MUL 
                self.eat(MUL)                                          #if yes, calls eat function 
                res = res * self.factor()                              #storing the result of res * factor function to the same varaible res
            else:
                self.eat(DIV)                                          #else calls eat function for DIV
                res = res / self.factor()                              #storing the result of res / factor function to the same varaible res
        return res                                                 #returning the res     
    
    def expr(self):                                             #defining the function expr
        
        result = self.term()                                       #calling the function term and storing it to the variable result

        while self.current_token.type_ in (PLUS, MINUS):               #checking if the type of the current token is PLUS or MINUS
            token = self.current_token                                   #if yes, stores the token to the token variable
            if token.type_ == PLUS:                                       #check got the token type is PLUS
                self.eat(PLUS)                                              #if yes, calls eat function for PLUS
                result = result + self.term()                               #storing the result of result + factor function to the same varaible result
            else:
                self.eat(MINUS)                                             #else calls eat function for MINUS
                result = result - self.term()                               #storing the result of result - factor function to the same varaible result
        return result                                             #returning the result

## Creating the Random Expression

In [29]:
import random                                                        #importing the library random
 
def random_expr(max_depth=3):                                        #defining the function random_Expr and making max_Depth = 3
    if max_depth == 0:                                               #checking the max_Depth = 0 
        return random_factor(max_depth)                                #if yes, returning the function random_factor(max_depth)
    
    expr = random_term(max_depth)                                    #calling the function random_term and storing its result to variable expr
    
    # Add more terms with random operators
    while random.choice([True, False]) and max_depth > 0:           #adding the terms till max_depth is greater than 0 and condition is True
        op = random.choice(['+', '-'])                               #if yes, randomly choosing the operator
        term = random_term(max_depth-1)                              #getting the next term
        expr += f" {op} {term}"                                      #adding the terms to the variable expr
    
    return expr                                                     #returning the variable expr

def random_term(max_depth):                                         #defining the function random_term  
    if max_depth == 0:                                              #checking the max_Depth = 0 
        return random_factor(max_depth)                                #if yes, returning the function random_factor(max_depth)
    
    term = random_factor(max_depth)                                 #calling the function random_factor and storing its result to variable term
    
    # Add more factors with random operators
    while random.choice([True, False]) and max_depth > 0:           #adding the factors till max_depth is greater than 0 and condition is True
        op = random.choice(['*', '/'])                               #if yes, randomly choosing the operator
        factor = random_factor(max_depth-1)                          #getting the next factor
        term += f" {op} {factor}"                                    #adding the terms to the variable term
    
    return term                                                    #returning the variable expr

def random_factor(max_depth):                                      #defining the function random_factor
    if max_depth == 0:                                             #checking the max_Depth = 0
        return str(random.randint(1, 1000))                         #if yes, returning the random number between 1 - 1000
    if random.choice([True, False]):                               #randomly checking the condition is True or False
        return str(random.randint(1, 1000))                         #if True, returns an string of number between 1 - 1000
    else:
        return f"({random_expr(max_depth-1)})"                    #else, calling the expr function inside the parathesis'(expr)'

## Calling the random Expression Function to Generate the Expressions

In [32]:
num = random.randint(3,10)                                   #randomly choosing the number if expression to be generated
for i in range(num):                                         #running for loop till range of num
    expr = random_expr(max_depth=3)                      #calling the function random_Expr and passing depth to 3
    print(f'\nThe Expression {i+1} is : {expr}')                                          #printing the expr 
    lexer = Lexer(expr)                            #creating the object for Lexer called lexer
    interpreter = Interpreter(lexer)                     #creating the object for Interpreter called interpreter
    res = interpreter.expr()                             #calling the expr function of interpreter and storing it in res variable
    print(f'The result of the above Expression is: {res}\n')                                           #printing the res 


The Expression 1 is : (612) / 379 + (554 * 220 * 313 - 481 - 150 - 178 + 547) * (500) / (584) * 508 * 62 + (166 * 249 - 249) * 944
The result of the above Expression is: 1028733145789.56


The Expression 2 is : 895 / (452 + 418 - 927)
The result of the above Expression is: -15.701754385964913


The Expression 3 is : (((564) / 921 / 548 + 60) - (619) + (937) * 235 - (463))
The result of the above Expression is: 219173.00111747783


The Expression 4 is : 744
The result of the above Expression is: 744

